In [ ]:
# python3 -m venv .venv
# source .venv/bin/activate
# pip install mellow_strategy_sdk

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import polars as pl
pd.set_option('display.max_colwidth', 70) # to fit hashes


from mellow_sdk.primitives import Pool, POOLS, MIN_TICK, MAX_TICK
from mellow_sdk.data import RawDataUniV3
from mellow_sdk.strategies import AbstractStrategy, UniV3Passive
from mellow_sdk.backtest import Backtest
from mellow_sdk.positions import BiCurrencyPosition, UniV3Position
from mellow_sdk.viewers import PortfolioViewer, UniswapViewer, RebalanceViewer

from IPython.display import Image

In [ ]:
# POOLS - dict with available pools 
POOLS[1]

In [ ]:
# choose WBTC/WETH 0.3% fee
pool_num = 1
pool = Pool(
    tokenA=POOLS[pool_num]['token0'],
    tokenB=POOLS[pool_num]['token1'],
    fee=POOLS[pool_num]['fee']
)

In [ ]:
# if there is no folder or files, create and download
data = RawDataUniV3(pool=pool, data_dir='data', reload_data=False).load_from_folder()

# V2 Passive

$ UniV2 = UniV3(0, \infty) $

In [ ]:
# create strategy from example, perform backtest

v2_strat = UniV3Passive(
    lower_price=1e-10,  #1.0001 ** MIN_TICK Unfortunately doesn't fit in data types,
    upper_price=1e10,  #1.0001 ** MAX_TICK Unfortunately doesn't fit in data types,
    pool=pool,
    gas_cost=0.01,
    name='passive_v2'
)

bt = Backtest(strategy=v2_strat)
portfolio_history, rebalance_history, uni_history = bt.backtest(df=data.swaps)

In [ ]:
# making plots on backtest results

rv = RebalanceViewer(rebalance_history)
uv = UniswapViewer(uni_history)
pv = PortfolioViewer(portfolio_history, pool)

# Draw portfolio stats, like value, fees earned, apy
fig1, fig2, fig3, fig4, fig5, fig6 = pv.draw_portfolio()

# Draw Uniswap intervals
intervals_plot = uv.draw_intervals(data.swaps)

# Draw rebalances
rebalances_plot = rv.draw_rebalances(data.swaps)

# Calculate df with portfolio stats
stats = portfolio_history.calculate_stats()

In [ ]:
# gAPY at last moment
stats['g_apy'][-1]

In [ ]:
# calculated portfolio statistics
stats.head(2)

In [ ]:
fig6.show()
fig6.write_image('v2_fig6.png')

# V3 Passive

In [ ]:
v3_strat = UniV3Passive(
#     lower_price=data.swaps['price'].min() + 1,
#     upper_price=data.swaps['price'].max() - 1,
    lower_price=15,
    upper_price=16,
    pool=pool,
    gas_cost=0.01,
    name='passive_v3'
)

bt = Backtest(strategy=v3_strat)
portfolio_history, rebalance_history, uni_history = bt.backtest(df=data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
uv = UniswapViewer(uni_history)
pv = PortfolioViewer(portfolio_history, pool)

# Draw portfolio stats, like value, fees earned, apy
fig1, fig2, fig3, fig4, fig5, fig6 = pv.draw_portfolio()

# Draw Uniswap intervals
intervals_plot = uv.draw_intervals(data.swaps)

# Draw rebalances
rebalances_plot = rv.draw_rebalances(data.swaps)

# Calculate df with portfolio stats
stats = portfolio_history.calculate_stats()

In [ ]:
stats['g_apy'][-1]

In [ ]:
fig6.show()
fig6.write_image('v3_fig6.png')

# Catch the price strategy

In [ ]:
# create new strategy, should be inherited from AbstractStrategy

class StrategyCatchThePrice(AbstractStrategy):
    """
    ``UniV3Passive`` is the passive strategy on UniswapV3 without rebalances.
        lower_price: Lower bound of the interval
        upper_price: Upper bound of the interval
        rebalance_cost: Rebalancing cost, expressed in currency
        pool: UniswapV3 Pool instance
        name: Unique name for the instance
    """

    def __init__(
        self,
        name: str,
        pool: Pool,
        gas_cost: float,
        width: int,
        seconds_to_hold: int
    ):
        super().__init__(name)
        self.fee_percent = pool.fee.percent
        self.gas_cost = gas_cost
        self.swap_fee = pool.fee.percent
        
        self.width = width
        self.seconds_to_hold = seconds_to_hold
        
        self.last_mint_price = None
        self.last_timestamp_in_interval = None
        self.pos_num = None

        
    def create_pos(self, x_in, y_in, price, timestamp, portfolio):
        """
            Swaps x_in, y_in in right proportion and mint to new interval
        """
        if self.pos_num is None:
            self.pos_num = 1
        else:
            self.pos_num += 1
            
        # bicurrency position that can swap tokens
        bi_cur = portfolio.get_position('main_vault')
        
        # add tokens to bicurrency position
        bi_cur.deposit(x_in, y_in)
        
        # new uni position
        uni_pos = UniV3Position(
            name=f'UniV3_{self.pos_num}', 
            lower_price=max(1.0001 ** MIN_TICK, price - self.width), 
            upper_price=min(1.0001 ** MAX_TICK, price + self.width), 
            fee_percent=self.fee_percent, 
            gas_cost=self.gas_cost
        )
        
        # add new position to portfolio
        portfolio.append(uni_pos)
        
        # uni_pos.aligner is UniswapLiquidityAligner, good class for working with liquidity operations
        dx, dy = uni_pos.aligner.get_amounts_for_swap_to_optimal(
            x_in, y_in, swap_fee=bi_cur.swap_fee, price=price
        )
        
        # swap tokens to right proportion (if price in interval swaps to equal liquidity in each token)
        if dx > 0:
            bi_cur.swap_x_to_y(dx, price=price)
        if dy > 0:
            bi_cur.swap_y_to_x(dy, price=price)

        x_uni, y_uni = uni_pos.aligner.get_amounts_after_optimal_swap(
            x_in, y_in, swap_fee=bi_cur.swap_fee, price=price
        )
        
        # withdraw tokens from bicurrency
        # because of float numbers precision subtract 1e-9
        bi_cur.withdraw(x_uni - 1e-9, y_uni - 1e-9)
        
        # deposit tokens to uni
        uni_pos.deposit(x_uni, y_uni, price=price)
        
        # remember last mint price to track price in interval
        self.last_mint_price = price
        
        # remember timestamp price was in interval
        self.last_timestamp_in_interval = timestamp

    def rebalance(self, *args, **kwargs) -> str:
        """
            Function of AbstractStrategy
            In Backtest.backtest this function process every row of historic data
            
            Return: name of portfolio action, that will be processed by RebalanceViewer
        """
        # record is row of historic data
        record = kwargs['record']
        timestamp = record['timestamp']
        event = record['event']
        
        # portfolio managed by the strategy
        portfolio = kwargs['portfolio']
        price_before, price = record['price_before'], record['price']
        
        # process only swap events
        if event != 'swap':
            return None
        
        if len(portfolio.positions) == 0:
            # create biccurency positions for swap
            bi_cur = BiCurrencyPosition(
                name=f'main_vault',
                swap_fee=self.swap_fee,
                gas_cost=self.gas_cost,
                x=0,
                y=0,
                x_interest=None,
                y_interest=None
            )
            portfolio.append(bi_cur)
            
            # create first uni interval
            self.create_pos(x_in=1/price, y_in=1, price=price, timestamp=timestamp, portfolio=portfolio)
            return 'init'
        
        # collect fees from uni
        uni_pos = portfolio.get_position(f'UniV3_{self.pos_num}')
        uni_pos.charge_fees(price_0=price_before, price_1=price)

        
        # if price in interval update last_timestamp_in_interval
        if abs(self.last_mint_price - price) < self.width:
            self.last_timestamp_in_interval = timestamp
            return None
        
        # if price outside interval for long create new uni position
        if (timestamp - self.last_timestamp_in_interval).total_seconds() > self.seconds_to_hold:
            uni_pos = portfolio.get_position(f'UniV3_{self.pos_num}')
            x_out, y_out = uni_pos.withdraw(price)
            portfolio.remove(f'UniV3_{self.pos_num}')
            self.create_pos(x_in=x_out, y_in=y_out, price=price, timestamp=timestamp, portfolio=portfolio)
            return 'rebalance'

        return None

In [ ]:
catch_strat = StrategyCatchThePrice(
    name='name',
    pool=pool,
    gas_cost=0, # in this strategy gas can eat all portfolio, for this example set 0
    width=0.5,
    seconds_to_hold=60*60
)

bt = Backtest(strategy=catch_strat)
portfolio_history, rebalance_history, uni_history = bt.backtest(df=data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
uv = UniswapViewer(uni_history)
pv = PortfolioViewer(portfolio_history, pool)

# Draw portfolio stats, like value, fees earned, apy
fig1, fig2, fig3, fig4, fig5, fig6 = pv.draw_portfolio()

# Draw Uniswap intervals
intervals_plot = uv.draw_intervals(data.swaps)

# Draw rebalances
rebalances_plot = rv.draw_rebalances(data.swaps)

# Calculate df with portfolio stats
stats = portfolio_history.calculate_stats()

In [ ]:
# number of rebalances
rv.rebalance_history.to_df().shape[0] 

In [ ]:
rebalances_plot.show()
rebalances_plot.write_image('catch_rebalances.png')

In [ ]:
intervals_plot.show()
intervals_plot.write_image('catch_intervals.png')

In [ ]:
fig2.show()

In [ ]:
fig2.show()
fig2.write_image('catch_fig2.png')

In [ ]:
fig6.show()
fig6.write_image('catch_fig6.png')

In [ ]:
stats.tail(2)

In [ ]:
# fast draw all in one cell
display(
    *[
        Image(i.update_layout(height=300,width=700).to_image(format='png')) 
        for i in [intervals_plot, rebalances_plot, fig2, fig4, fig6]
    ]
)